# House price regression

# 1. Import libraries

In [46]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, Ridge
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

from sklearn.model_selection import KFold, cross_validate

from scipy.stats import skew, boxcox_normmax
from scipy.special import boxcox1p, inv_boxcox1p

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings('ignore')

# 2. Preprocessing pipeline

Steps to include in the preprocessing pipeline:
- Dropping features
- Reformat data type into correct one
- Impute missing data
- Apply transformation on numerical data
- Feature engineering
- Encode categorical data

In [47]:
class BoxcoxTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, skew_threshold=0.5):
        self.lmbda = []
        self.skewed_features = []
        self.skew_threshold = skew_threshold

    def fit(self, X, y=None):
        skewed = X[X.dtypes[X.dtypes != "object"].index].apply(lambda x: skew(x.dropna().astype(float)))
        self.skewed_features = skewed[skewed > self.skew_threshold].index.to_list()
        for col in self.skewed_features:
            self.lmbda.append(boxcox_normmax(X[col] + 1))
        return self

    def transform(self, X, *_):
        X = pd.DataFrame(X)
        if X[self.skewed_features].shape[1] != len(self.lmbda):
            raise ValueError("X has %d features per sample, expected %d"
                             % (X[self.skewed_features].shape[1], len(self.lmbda)))
        i = 0
        for col in self.skewed_features:
            X[col] = boxcox1p(X[col], self.lmbda[i])
            i = i + 1
        return X

In [48]:
class Classic_imputer():
    def __init__(self, impute_0, impute_none, impute_med, impute_freq):
        self.impute_0 = impute_0
        self.impute_none = impute_none
        self.impute_med = impute_med
        self.impute_freq = impute_freq
        
        self.median_values = {}
        self.most_frequent_values = {}

    def fit(self, X, y=None):
        #save the median and most frequent value to not have data leakage
        for col in self.impute_med:
            self.median_values[col] = X[col].mode()[0]
        for col in self.impute_freq:
            self.most_frequent_values[col] = X[col].mode()[0]
        return self
    
    def transform(self, X):
        #fill the NA value based on the rule
        for col in self.impute_0:
            X[col] = X[col].fillna(0)
        for col in self.impute_none:
            X[col] = X[col].fillna('none')
        for col in self.impute_med:
            X[col] = X[col].fillna(self.median_values[col])
        for col in self.impute_freq:
            X[col] = X[col].fillna(self.most_frequent_values[col])
        return X

In [49]:
class DataFrameOneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, drop=None, handle_unknown="error"):
        self.categories = "auto"
        self.drop = drop
        self.dtype = dtype=np.float64
        self.handle_unknown = handle_unknown
        
    def fit(self, X, y=None):
        self.onehotencoders_ = []
        self.column_names_ = []

        for c in X.select_dtypes(include=["object", "category"]).columns:
            # Construct the OHE parameters using the arguments
            ohe_params = {
                "categories": self.categories,
                "drop": self.drop,
                "sparse": False,
                "dtype": self.dtype,
                "handle_unknown": self.handle_unknown,
            }
            
            # Now create, fit, and store the onehotencoder for current column c
            ohe = OneHotEncoder(**ohe_params)
            self.onehotencoders_.append(ohe.fit(X.loc[:, [c]]))

            # Get the feature names and replace each x0_ with empty and after that
            # surround the categorical value with [] and prefix it with the original
            # column name
            feature_names = ohe.get_feature_names()
            feature_names = [x.replace("x0_", "") for x in feature_names]
            feature_names = [f"{c}_{x}" for x in feature_names]

            self.column_names_.append(feature_names)

        return self

    def transform(self, X):
        all_df = []

        for i, c in enumerate(X.select_dtypes(include=["object", "category"]).columns):
            ohe = self.onehotencoders_[i]
            transformed_col = ohe.transform(X.loc[:, [c]])
            df_col = pd.DataFrame(transformed_col, columns=self.column_names_[i])
            all_df.append(df_col)

        return pd.concat([X.select_dtypes(exclude=["object", "category"]), pd.concat(all_df, axis=1)], axis=1)

In [50]:
class DataframeFunctionTransformer():
    def __init__(self, func):
        self.func = func

    def transform(self, input_df, **transform_params):
        return self.func(input_df)

    def fit(self, X, y=None, **fit_params):
        return self

In [51]:
def has_particularities(df):
    df["Has_shed"] = (df["MiscFeature"] == "Shed") * 1  
    df["Remodeled"] = (df["YearRemodAdd"] != df["YearBuilt"]) * 1
    df["Recent_remodel"] = (df["YearRemodAdd"] == df["YrSold"]) * 1
    df["Very_new_house"] = (df["YearBuilt"] == df["YrSold"]) * 1
    df["Has_2nd_floor"] = (df["2ndFlrSF"] != 0) * 1
    df["Has_pool"] = (df["PoolArea"] != 0) * 1
    df["Has_Wood_deck"] = (df["WoodDeckSF"] != 0) * 1
    return df

def reformat_categorical_data(df):
    df['MSSubClass'] = df['MSSubClass'].apply(str)
    df['MoSold'] = df['MoSold'].astype(str)
    return df

def new_features(df):
    df['TotalSF'] = df['1stFlrSF'] + df['2ndFlrSF'] + df['TotalBsmtSF']
    df['age'] = df['YrSold'] - df['YearBuilt']
    df['Total_bath'] = df['FullBath'] + 0.5*df['HalfBath'] + 
                       df['BsmtFullBath'] + 0.5*df['BsmtHalfBath']
    return df

def drop_features(df):
    features = ['Utilities', 'Street', 'PoolQC', 'Id']
    df_ = df.drop(columns = features)
    return df_

In [52]:
feature_engineering = Pipeline([
    ('has_particularities' , DataframeFunctionTransformer(has_particularities)),
    ('new_features' , DataframeFunctionTransformer(new_features))  
])

reformat_data_type = Pipeline([
    ('reformat data type' , DataframeFunctionTransformer(reformat_categorical_data))
])

In [53]:
impute_0 = ['BsmtFullBath', 'BsmtHalfBath', 'TotalBsmtSF', 'BsmtFinSF2', 'GarageYrBlt', 'GarageArea', 'GarageCars', 
                          'MasVnrArea', 'BsmtUnfSF', 'BsmtFinSF1']
impute_none = ['MSSubClass', 'HeatingQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 
                             'GarageCond', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtFinType1', 'BsmtExposure', 'BsmtFinType2', 'MasVnrType', 'KitchenQual']
impute_med = ['LotFrontage']
impute_freq = ['SaleType', 'Electrical', 'Exterior1st', 'Exterior2nd', 'MSZoning', 'Functional']

In [54]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

y = np.log1p(np.ravel(np.array(train['SalePrice']).T))
X = train.drop(columns=['SalePrice'])

X_pred = test

folds = KFold(n_splits=10, shuffle=True, random_state=0)

In [59]:
pipeline = Pipeline([
    ('drop' , DataframeFunctionTransformer(drop_features)),
    ('data type' , reformat_data_type),
    ('imputer' , Classic_imputer(impute_0, impute_none, impute_med, impute_freq)),
    ('boxcox transformer' , BoxcoxTransformer(skew_threshold = 0.5)),
    ('feature_engineering', feature_engineering),
    ('encoding' , DataFrameOneHotEncoder(handle_unknown="ignore"))
])
pipeline.fit(X)
X_preprocessed = pipeline.transform(X)
X_pred_preprocessed = pipeline.transform(X_pred)

In [56]:
X

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029,0,1078,GasA,Gd,Y,FuseA,1078,0,0,1078,1,0,1,0,2,1,Gd,5,Typ,0,NaN,Attchd,1950.0,Un

In [57]:
X_preprocessed

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,YrSold,Has_shed,Remodeled,Recent_remodel,Very_new_house,Has_2nd_floor,Has_pool,Has_Wood_deck,TotalSF,age,Total_bath,MSSubClass_120,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Alley_Grvl,Alley_Pave,Alley_none,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,MasVnrType_none,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtQual_none,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtCond_none,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_none,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType1_none,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,BsmtFinType2_none,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Func

In [60]:
(X_preprocessed.shape, X_pred_preprocessed.shape)

((1460, 330), (1459, 330))

# 3. Prediction pipeline

In [61]:
models_list = {'Ridge': Ridge(),
               'SVR': SVR(), 
               'LGBMRegressor' :LGBMRegressor(verbosity = 0, force_row_wise=True), 
               'Lasso': Lasso(alpha=0.0005),
               'ElasticNet': ElasticNet(alpha=0.0006, l1_ratio=1),
               'BayesianRidge': BayesianRidge(),
               'GradientBoostingRegressor': GradientBoostingRegressor()
              }

scoring = {'r2':'r2'}
columns = ['Model', 'Median fit time', 'Mean r2', 'Std r2']

x = X_preprocessed.to_numpy()

model_perf_matrix = []
predictions = pd.DataFrame()
for model_name, model in models_list.items():
    pipeline = Pipeline([
        ('model' , model)
    ])

    cv_score = cross_validate(pipeline, x, y, cv=folds, scoring=scoring, 
                              verbose=0, error_score="raise");
    model_perf_matrix.append([model_name, round(cv_score['fit_time'].mean(),3),
                              round(cv_score['test_r2'].mean(),4), round(cv_score['test_r2'].std(),4)])
    
    pipeline.fit(x,y)
    predictions[model_name] = np.floor(np.expm1(pipeline.predict(X_pred_preprocessed.to_numpy()))).T
    
df_model_perf = pd.DataFrame(model_perf_matrix, columns=columns)
df_model_perf

,Model,Median fit time,Mean r2,Std r2
0,Ridge,0.085,0.8844,0.0642
1,SVR,0.173,0.7280,0.0794
2,LGBMRegressor,0.305,0.8900,0.0288
3,Lasso,0.373,0.8953,0.0674
4,ElasticNet,0.367,0.8956,0.0651
5,BayesianRidge,0.191,0.8894,0.0595
6,GradientBoostingRegressor,1.262,0.8959,0.0228


In [62]:
predictions.head(20)

,Ridge,SVR,LGBMRegressor,Lasso,ElasticNet,BayesianRidge,GradientBoostingRegressor
0,119623.0,161666.0,118175.0,120386.0,120022.0,118193.0,125077.0
1,154638.0,155360.0,156227.0,154571.0,154046.0,153248.0,155271.0
2,182237.0,193833.0,185593.0,178726.0,178674.0,177547.0,182463.0
3,199648.0,190984.0,191634.0,200402.0,200746.0,198346.0,185443.0
4,197117.0,171990.0,184297.0,197528.0,197242.0,195343.0,193855.0
5,169855.0,183005.0,172719.0,170255.0,170753.0,167970.0,175205.0
6,179866.0,169719.0,174842.0,182020.0,182911.0,180899.0,173171.0
7,161318.0,161024.0,166343.0,161568.0,161751.0,161972.0,170244.0
8,192723.0,179235.0,179259.0,194894.0,195358.0,189716.0,186916.0
9,122699.0,149334.0,119846.0,120127.0,119634.0,122208.0,128569.0
